<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fa2f65f835df17b01aeda19f904fe9eb04c4ce1f1dbf2b1896cf5cf02ab991e5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-18 16:09:54
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: -4.52 K (-0.03%)
Current PnL: -27.81 L (-17.96%)
CY Booked + Current PnL: -13.52 L (-8.73%)
-------------------
Total profit:  1.38 L
Total loss:  -29.19 L
-------------------
Total Booked + Current PnL: 13.46 L (10.53%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.26%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.44 L (66.39%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.54,9.66,9.15,19.69,19238.0,18513.0,210255.0,-1.08,59.0,X-MC,2.52,78.0,0.96,1.52,17.11,XY25,NTT,AC
77,TTKPRESTIG,770.00,-1.21,-20.15,25.35,0.09,20400.0,-20304.0,80473.0,86.77,27.0,M-SC,2.38,253.0,-1.00,0.58,5.72,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.81,-4.32,27.56,22.05,25838.0,-4230.0,93750.0,-17.34,48.0,H-SC,6.77,164.0,-0.16,0.68,36.27,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.63,-16.27,19.45,0.02,28428.0,-28400.0,146159.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.05,27.13,OX40N,NTT,PAINTS
43,ITC,452.00,0.15,-1.86,12.89,10.78,30399.0,-4482.0,235836.0,-41.56,43.0,X-LC,2.06,5.0,-0.15,1.70,3.27,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,7.26,-44.60,205.14,69.05,151229.0,-59346.0,73720.0,-55.25,59.0,H-SC,22.33,140.0,-0.39,0.53,13.69,SR,ATH,IT
6,ASIANTILES,137.00,6.66,-8.92,99.48,81.67,84994.0,-8372.0,85438.0,7688.89,64.0,L-SC,12.50,271.0,-0.10,0.62,65.45,XR,NTT,CERAMICS
15,CAMS,950.00,2.52,-1.43,26.28,24.48,65242.0,-3597.0,248259.0,-81.09,45.0,X-SC,0.56,86.0,-0.06,1.79,21.38,X40N,NTT,MISC
36,ICICIPRULI,790.00,2.40,6.84,22.36,30.73,47064.0,13477.0,210482.0,-18.17,60.0,X-MC,2.43,75.0,0.29,1.52,20.48,X40,ATH,INSURANCE
71,TANLA,1943.92,2.31,-36.23,244.51,119.69,422171.0,-98101.0,172660.0,-35.52,46.0,H-SC,7.81,156.0,-0.23,1.25,36.40,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-3.44,-63.50,174.16,0.07,87200.0,-87108.0,50069.0,1734.60,50.0,L-SC,2.35,268.0,-1.00,0.36,24.52,OX40N,NTT,JEWELLERY
28,GREENPANEL,537.0,-3.26,-39.89,138.13,43.14,152011.0,-73029.0,110049.0,146.77,27.0,M-SC,10.15,240.0,-0.48,0.79,3.52,XY24,NTT,MISC
80,VALIANTORG,838.0,-2.80,-46.21,223.74,74.15,196911.0,-75596.0,88009.0,-258.45,39.0,H-SC,22.87,149.0,-0.38,0.64,12.81,XR,NTT,CHEMICALS
16,CERA,9475.0,-2.58,-30.82,79.08,23.89,96235.0,-54210.0,121693.0,-33.17,39.0,H-SC,11.07,157.0,-0.56,0.88,5.10,OX40N,NTT,CERAMICS
78,UNITDSPR,1644.0,-2.37,2.20,18.17,20.77,43478.0,5142.0,239286.0,-6.70,40.0,X-MC,2.34,62.0,0.12,1.73,9.27,X40N,NTT,BREWERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.12,-1.87,111.87,107.91,162086.0,-2756.0,144888.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.05,8.03,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.63,-16.27,19.45,0.02,28428.0,-28400.0,146159.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.05,27.13,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.21,-20.15,25.35,0.09,20400.0,-20304.0,80473.0,86.77,27.0,M-SC,2.38,253.0,-1.00,0.58,5.72,OX40N,NTT,DURABLES
66,SIS,528.00,-0.05,-24.25,60.07,21.26,50526.0,-26920.0,84112.0,1984.46,50.0,H-SC,4.96,166.0,-0.53,0.61,13.64,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.52,-26.27,53.91,13.47,107183.0,-70848.0,198819.0,-69.67,31.0,H-SC,6.06,158.0,-0.66,1.43,1.09,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.04,8.22,59.18,72.27,108522.0,13928.0,183376.0,-6.99,69.0,M-LC,2.81,99.0,0.13,1.32,14.9,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.04,8.22,59.18,72.27,108522.0,13928.0,183376.0,-6.99,69.0,M-LC,2.81,99.0,0.13,1.32,14.90,XR,NTT,IT
25,FINCABLES,1641.55,1.12,-1.87,111.87,107.91,162086.0,-2756.0,144888.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.05,8.03,OX40N,ATH,CABLES
38,INDIAMART,4810.62,1.21,-2.71,112.48,106.72,134967.0,-3344.0,119992.0,-51.77,42.0,H-SC,3.41,139.0,-0.02,0.87,20.03,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.48,-3.65,40.77,35.63,82322.0,-7657.0,201917.0,-17.18,36.0,H-MC,2.77,119.0,-0.09,1.46,12.41,AR,ATH,PHARMA
37,IEX,219.00,-0.33,-5.15,56.70,48.63,107534.0,-10300.0,189654.0,-36.95,45.0,H-SC,13.55,136.0,-0.10,1.37,5.81,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-1.00,-11.39,44.57,28.11,79546.0,-22935.0,178475.0,-35.73,24.0,X-MC,13.48,55.0,-0.29,1.29,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.48,-18.59,90.97,55.47,180178.0,-45223.0,198063.0,-27.82,26.0,X-MC,13.92,64.0,-0.25,1.43,1.48,X40N,ATH,FINANCE
8,AWL,485.00,-0.91,-24.28,102.72,53.50,234699.0,-73267.0,228484.0,-63.37,28.0,X-MC,3.78,58.0,-0.31,1.65,1.48,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.08,-39.36,122.96,35.20,95943.0,-50642.0,78028.0,2.24,30.0,X-SC,16.44,92.0,-0.53,0.56,0.09,X40,NTT,FOOTWEAR
23,DMART,5391.45,-1.80,-9.68,43.50,29.60,98063.0,-24168.0,225432.0,-22.96,30.0,X-LC,6.37,19.0,-0.25,1.63,10.38,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,2.52,-1.43,26.28,24.48,65242.0,-3597.0,248259.0,-81.09,45.0,X-SC,0.56,86.0,-0.06,1.79,21.38,X40N,NTT,MISC
12,BERGEPAINT,680.0,-0.47,-4.45,26.78,21.14,58172.0,-10117.0,217222.0,-11.18,38.0,X-MC,1.72,74.0,-0.17,1.57,22.44,XY24,NTT,PAINTS
20,DABUR,735.0,-0.32,-3.70,49.22,43.70,117580.0,-9181.0,238887.0,-11.62,34.0,X-MC,1.98,72.0,-0.08,1.72,10.79,XY24,BTT,FMCG
43,ITC,452.0,0.15,-1.86,12.89,10.78,30399.0,-4482.0,235836.0,-41.56,43.0,X-LC,2.06,5.0,-0.15,1.70,3.27,X40,NTT,FMCG
78,UNITDSPR,1644.0,-2.37,2.20,18.17,20.77,43478.0,5142.0,239286.0,-6.70,40.0,X-MC,2.34,62.0,0.12,1.73,9.27,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.23,-26.25,122.58,64.15,215116.0,-62461.0,175490.0,-56.75,30.0,X-SC,3.01,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
53,PAGEIND,51605.07,-1.00,-11.39,44.57,28.11,79546.0,-22935.0,178475.0,-35.73,24.0,X-MC,13.48,55.0,-0.29,1.29,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.08,-39.36,122.96,35.20,95943.0,-50642.0,78028.0,2.24,30.0,X-SC,16.44,92.0,-0.53,0.56,0.09,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,0.12,-20.64,78.32,41.51,163689.0,-54365.0,209000.0,-5.42,35.0,X-MC,7.50,61.0,-0.33,1.51,0.12,XY25,ATH,FMCG
75,TMPV,600.00,-0.16,-29.12,73.51,22.98,117693.0,-65793.0,160105.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.16,0.70,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.34,-30.70,106.12,42.84,47807.0,-19956.0,45050.0,-54.82,40.0,X-SC,35.16,83.0,-0.42,0.33,1.92,XY24,NTT,MISC
59,RELAXO,1176.00,0.36,-47.85,192.03,52.31,145382.0,-69452.0,75708.0,-44.44,42.0,X-SC,7.22,91.0,-0.48,0.55,1.34,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.08,-39.36,122.96,35.20,95943.0,-50642.0,78028.0,2.24,30.0,X-SC,16.44,92.0,-0.53,0.56,0.09,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.87,-3.76,28.93,24.08,35346.0,-4774.0,122176.0,-11.59,37.0,X-SC,5.31,89.0,-0.14,0.88,15.12,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.64,-17.42,73.55,43.32,98925.0,-28372.0,134500.0,-29.05,33.0,X-SC,5.02,90.0,-0.29,0.97,3.27,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.96,-9.79,33.81,20.71,115361.0,-37034.0,341203.0,-22.33,69.0,X-LC,4.05,1.0,-0.32,2.46,14.01,X40,ATH,IT
41,INFY,1972.00,1.55,10.74,21.22,34.24,73529.0,33620.0,346508.0,-13.85,66.0,X-LC,4.58,2.0,0.46,2.50,19.92,X40,NTT,IT
75,TMPV,600.00,-0.16,-29.12,73.51,22.98,117693.0,-65793.0,160105.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.16,0.70,XY24,NTT,AUTO
81,VBL,671.64,0.58,-4.33,41.82,35.67,126362.0,-13685.0,302157.0,-15.77,52.0,X-LC,4.04,4.0,-0.11,2.18,8.92,X40N,ATH,FMCG
43,ITC,452.00,0.15,-1.86,12.89,10.78,30399.0,-4482.0,235836.0,-41.56,43.0,X-LC,2.06,5.0,-0.15,1.70,3.27,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6.66,-8.92,99.48,81.67,84994.0,-8372.0,85438.0,7688.89,64.0,L-SC,12.50,271.0,-0.10,0.62,65.45,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.81,-4.32,27.56,22.05,25838.0,-4230.0,93750.0,-17.34,48.0,H-SC,6.77,164.0,-0.16,0.68,36.27,XR,ATH,FINANCE
13,BSOFT,831.70,0.35,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,65.0,H-SC,6.60,151.0,-0.30,0.80,28.66,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.63,-16.27,19.45,0.02,28428.0,-28400.0,146159.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.05,27.13,OX40N,NTT,PAINTS
84,WIPRO,420.00,1.04,8.22,59.18,72.27,108522.0,13928.0,183376.0,-6.99,69.0,M-LC,2.81,99.0,0.13,1.32,14.90,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.35,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,65.0,H-SC,6.60,151.0,-0.30,0.80,28.66,XR,ATH,IT
84,WIPRO,420.00,1.04,8.22,59.18,72.27,108522.0,13928.0,183376.0,-6.99,69.0,M-LC,2.81,99.0,0.13,1.32,14.90,XR,NTT,IT
41,INFY,1972.00,1.55,10.74,21.22,34.24,73529.0,33620.0,346508.0,-13.85,66.0,X-LC,4.58,2.0,0.46,2.50,19.92,X40,NTT,IT
73,TCS,4389.97,1.96,-9.79,33.81,20.71,115361.0,-37034.0,341203.0,-22.33,69.0,X-LC,4.05,1.0,-0.32,2.46,14.01,X40,ATH,IT
61,ROUTE,2227.21,7.26,-44.60,205.14,69.05,151229.0,-59346.0,73720.0,-55.25,59.0,H-SC,22.33,140.0,-0.39,0.53,13.69,SR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.87
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.84
MC    30.94
LC    24.24
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.57
X40      23.18
X40N     13.02
XR        9.79
XY25      9.11
AR        9.02
OX40N     7.54
SR        1.02
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.47
H-SC    23.73
X-LC    20.78
M-SC    11.58
X-SC     8.08
H-MC     4.79
L-SC     1.45
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.33,-8.58,43.50
IT,13.23,-14.68,72.34
FINANCE,9.44,-21.80,74.78
MISC,7.29,-28.73,81.39
ELECTRICAL,5.97,-13.47,55.04
PAINTS,5.80,-14.69,31.73
INSURANCE,4.82,-1.50,36.24
PHARMA,4.14,-2.37,34.59
AUTO,2.81,-34.42,81.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3238748.0,21
AR,1324021.0,10
XR,1307545.0,13
X40,1008141.0,14
X40N,882140.0,9
OX40N,747838.0,10
XY25,378916.0,6
SR,279326.0,2
MH,77659.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3694594.0,25
M-SC,1452649.0,15
X-MC,1435500.0,16
X-LC,844663.0,11
X-SC,800127.0,8
H-MC,415746.0,3
L-SC,260881.0,3
M-LC,108522.0,1
H-LC,79878.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1300405.0      6
           AR         899218.0      5
M-SC       XY24       833200.0      6
H-SC       XR         793959.0      7
X-MC       X40        493956.0      7
           XY24       410451.0      3
           X40N       348166.0      4
H-SC       OX40N      344027.0      4
X-LC       X40        321876.0      5
M-SC       OX40N      316611.0      5
X-SC       X40N       309549.0      3
           XY24       298269.0      3
H-SC       SR         279326.0      2
X-LC       X40N       224425.0      2
H-MC       AR         218462.0      2
X-LC       XY24       199139.0      2
H-MC       XY24       197284.0      1
X-SC       X40        192309.0      2
X-MC       XY25       182927.0      2
M-SC       XR         176375.0      2
L-SC       XR         173681.0      2
M-SC       AR         126463.0      2
M-LC       XR         108522.0      1
X-LC       XY25        99223.0      2
L-SC       OX40N       87200.0      1
H-LC       AR          79878.0      1
H-SC       MH          77659.0      1
M-MC       XY25        59942.0      1
L-MC       XR          55008.0      1
L-LC       XY25        36824.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
